In [7]:
import numpy as np

# Number of bits
N = 4

# Generate all possible combinations of inputs
inputs = np.array(np.meshgrid(*[range(2)]*(N*2+1))).T.reshape(-1, N*2+1)

# Initialize the outputs
outputs = np.zeros((2**9, N+1), dtype=int)

# Compute the outputs for each input
for i, x in enumerate(inputs):
    # Extract A, B, and CIN from x
    A = x[:N][::-1]
    B = x[N:-1][::-1]
    CIN = x[-1]
    
    # Perform the addition
    CARRY = CIN
    for j in range(N):
        outputs[i, j] = A[j] ^ B[j] ^ CARRY
        CARRY = (A[j] & B[j]) | (A[j] & CARRY) | (B[j] & CARRY)
    
    # The final carry out is the last bit of the sum
    outputs[i, -1] = CARRY

# Reverse the bits to match the input order
outputs = outputs[:, ::-1]

print('Inputs:', inputs)
print('Outputs:', outputs)


Inputs: [[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 1 1 ... 1 1 1]
 [1 0 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]
Outputs: [[0 0 0 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 ...
 [1 0 1 1 1]
 [1 1 0 1 1]
 [1 1 1 1 1]]


In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# Step 3: Create a dataset for a 4-bit full adder

# Number of bits
N = 4

# Generate all possible combinations of inputs
inputs = np.array(np.meshgrid(*[range(2)]*(N*2+1))).T.reshape(-1, N*2+1)

# Initialize the outputs
outputs = np.zeros((2**9, N+1), dtype=int)

# Compute the outputs for each input
for i, x in enumerate(inputs):
    # Extract A, B, and CIN from x
    A = x[:N][::-1]
    B = x[N:-1][::-1]
    CIN = x[-1]
    
    # Perform the addition
    CARRY = CIN
    for j in range(N):
        outputs[i, j] = A[j] ^ B[j] ^ CARRY
        CARRY = (A[j] & B[j]) | (A[j] & CARRY) | (B[j] & CARRY)
    
    # The final carry out is the last bit of the sum
    outputs[i, -1] = CARRY

# Reverse the bits to match the input order
outputs = outputs[:, ::-1]

# Combine inputs and outputs
input_data = np.concatenate([inputs, outputs], axis=1)

# Step 4 and 5: Build and train a Binary Variational Autoencoder (VAE)

# Define the parameters
original_dim = 2*N+1 + N+1 # input dimension
intermediate_dim = 50 # size of the hidden layers
latent_dim = 20 # size of the latent space

# Define the Encoder
inputs_layer = tf.keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs_layer)
z_mean = layers.Dense(latent_dim)(h)
z_log_var = layers.Dense(latent_dim)(h)

# Define the sampling function
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.shape(z_mean)[0], latent_dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Use the sampling function to create a layer
z = layers.Lambda(sampling)([z_mean, z_log_var])

# Create the Encoder model
encoder = tf.keras.Model(inputs_layer, [z_mean, z_log_var, z], name='encoder')

# Define the Decoder
latent_inputs = tf.keras.Input(shape=(latent_dim,))
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs_layer = layers.Dense(original_dim, activation='sigmoid')(x)

# Create the Decoder model
decoder = tf.keras.Model(latent_inputs, outputs_layer, name='decoder')

# Define the VAE as a model with a custom train_step
class VAE(tf.keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.keras.losses.binary_crossentropy(data, reconstruction)
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

# Instantiate the VAE model
vae = VAE(encoder, decoder)

# Compile and train the VAE

vae.compile(optimizer=tf.keras.optimizers.Adam())
vae.fit(input_data, epochs=300, batch_size=32)


Epoch 1/300
16/16 [==============================] - 2s 2ms/step - loss: 2.4398 - reconstruction_loss: 0.7327 - kl_loss: 1.3863
Epoch 2/300
16/16 [==============================] - 0s 3ms/step - loss: 1.4173 - reconstruction_loss: 0.7237 - kl_loss: 0.6249
Epoch 3/300
16/16 [==============================] - 0s 2ms/step - loss: 1.1444 - reconstruction_loss: 0.7147 - kl_loss: 0.3966
Epoch 4/300
16/16 [==============================] - 0s 2ms/step - loss: 1.0294 - reconstruction_loss: 0.7197 - kl_loss: 0.2928
Epoch 5/300
16/16 [==============================] - 0s 2ms/step - loss: 0.9557 - reconstruction_loss: 0.7125 - kl_loss: 0.2286
Epoch 6/300
16/16 [==============================] - 0s 3ms/step - loss: 0.9061 - reconstruction_loss: 0.7123 - kl_loss: 0.1847
Epoch 7/300
16/16 [==============================] - 0s 3ms/step - loss: 0.8663 - reconstruction_loss: 0.7078 - kl_loss: 0.1530
Epoch 8/300
16/16 [==============================] - 0s 2ms/step - loss: 0.8390 - reconstruction_loss: 0

In [13]:
def generate_test_vectors(vae, num_samples):
    # Sample random points in the latent space
    random_latent_vectors = tf.random.normal(shape=(num_samples, latent_dim))
    
    # Decode them to fake data (test vectors)
    generated_data = vae.decoder.predict(random_latent_vectors)
    
    # Binarize the data
    generated_data = np.where(generated_data > 0.5, 1, 0)

    return generated_data

# Generate some test vectors
num_samples = 10  # define how many samples you want to generate
generated_data = generate_test_vectors(vae, num_samples)

# Print the generated test vectors
print(generated_data)


1/1 [==============================] - 0s 145ms/step
[[0 0 0 1 0 0 0 0 0 0 0 1 1 1]
 [0 1 0 0 0 1 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 0 0 0 0 0 0 0 0 1]
 [0 1 1 1 1 0 1 0 0 1 0 1 0 1]
 [0 1 0 1 0 1 0 0 0 1 0 0 0 0]
 [1 0 1 0 1 1 1 1 0 1 1 0 0 1]
 [1 1 1 0 1 0 1 1 1 1 0 1 1 1]
 [1 1 1 0 1 0 1 1 0 0 0 0 1 1]
 [0 1 0 0 0 1 1 0 0 0 1 0 1 0]
 [0 0 0 1 1 1 1 0 1 0 0 1 0 1]]
